In [1]:
suppressPackageStartupMessages({
library(tidyverse)
library(Gviz)
library(rtracklayer)
library(GenomicFeatures)
library(data.table)
    })

In [2]:
GTF_dir <- "../reference_files/gencode.v40.annotation.gtf"


## comment back in if you need to make txdb
#hg38_txdb <- makeTxDbFromGFF(GTF_dir,
#                                 format="gtf",
#                                 organism="Homo sapiens",
#                                 dbxrefTag = "gene_name")
 
 
#saveDb(hg38_txdb, "~/BE_paper_2022/reference_files/hg38.txdb")

hg38_txdb <- loadDb("../reference_files/hg38.txdb")


In [3]:
gtrack <- GenomeAxisTrack()
# specify the start and end of chromosome 
chr_no <- 12 
chr_start <- 9760000  # start of region
chr_end <- 9768000 # end of region

gen <- 'hg38'

## set gene region track
gtTrack <- GeneRegionTrack(hg38_txdb,
                           chromosome=chr_no, # chromosome number
                           start=chr_start, # start of region
                           end=chr_end, # end of region
                           transcriptAnnotation="gene_id", # symbol is the gene symbol
                           fontsize.group=20# free to adjust font size
)

In [8]:

bedgraph_enformer <- fread(
  "../data_and_analyses/Enformer_scores/CD69_original_enformer_Fig1D_locus/CD69.pooled.bedGraph.pooled.bedGraph",
  col.names = c('chromosome', 'start', 'end', 'value')
)


## here changed DA regions to bed format
## here change da windows format to bed, then filter to keep only specific ROI
system("tail -n+2 ../data_and_analyses/ATAC/GSE155555/stim_v_ctrl_csaw_DA-windows_all.txt | awk '{OFS=\"\t\"}{print $1,$2,$3,$13}' > ../data_and_analyses/ATAC/GSE155555/stim_v_ctrl_csaw_DA-windows_all.bed")
system("awk '$4 <= 0.20' ../data_and_analyses/ATAC/GSE155555/stim_v_ctrl_csaw_DA-windows_all.bed > ../data_and_analyses/ATAC/GSE155555/stim_v_ctrl_csaw_DA-windows.filtq2.bed")

jurkat_DA <- import.bed("../data_and_analyses/ATAC/GSE155555/stim_v_ctrl_csaw_DA-windows.filtq2.bed")
guides_CD69 <- import.bed("../data_and_analyses/guides_and_perturbation_results/dCas9_KRAB_guides.bed")

ctrl_bw_jurkat = import.bw("../data_and_analyses/ATAC/GSE155555/bigwigs/Jurkat_WT_1_none_unspecified.merged.bw",as="GRanges")
stim_bw_jurkat = import.bw("../data_and_analyses/ATAC/GSE155555/bigwigs/Jurkat_WT_1_anti_CD3_PMA_4h_unspecified.merged.bw",as="GRanges")
ctrl_bw_cd4 = import.bw("../data_and_analyses/ATAC/GSE124867/CD4_resting.merged.bw", as="GRanges")
stim_bw_cd4 = import.bw("../data_and_analyses/ATAC/GSE124867/CD4_activated.merged.bw", as="GRanges")

options(ucscChromosomeNames=FALSE)
IG_bedgraph <- DataTrack(
  range = bedgraph_enformer,
  type = "a",
  genome = 'hg38',
  chromosome=chr_no,
  start=chr_start, # start of region
  end=chr_end, # end of region
  name = "Input*Gradient",
    fill.histogram = "red", 
    alpha=1
)

jurkat_DA_regions <- AnnotationTrack(
  range = jurkat_DA,
  chromosome=chr_no,
  type = "a",
  start=chr_start, # start of region
  end=chr_end
)


CD69_guides <- AnnotationTrack(
  range = guides_CD69,
  chromosome=chr_no,
  type = "a",
  start=chr_start, # start of region
  end=chr_end # end of region
)


jurkat_ctrl_track <- DataTrack(
  range = ctrl_bw_jurkat,
  chromosome=chr_no,
  start=chr_start, # start of region
  end=chr_end, # end of region
  col.histogram = "red"
)

jurkat_stim_track <- DataTrack(
  range = stim_bw_jurkat,
  chromosome=chr_no,
  start=chr_start, # start of region
  end=chr_end, # end of region
  col.histogram = "red"
)

cd4_ctrl_track <- DataTrack(
  range = ctrl_bw_cd4,
  chromosome=chr_no,
  start=chr_start, # start of region
  end=chr_end, # end of region
  col.histogram = "red"
)

cd4_stim_track <- DataTrack(
  range = stim_bw_cd4,
  chromosome=chr_no,
  start=chr_start, # start of region
  end=chr_end, # end of region
  col.histogram = "red"
)

In [15]:
pdf("cd69_locus_plot_bigwigs.pdf")
## modify remainder in AI 
plotTracks(list(gtrack,
                gtTrack,
                cd4_ctrl_track,
                cd4_stim_track,
                jurkat_ctrl_track,
                jurkat_stim_track,
                jurkat_DA_regions,
                CD69_guides,
                IG_bedgraph), shape = "arrow", 
           transcriptAnnotation = "symbol", chromosome = "chr12",from = 9732000, to= 9776000,
           type = c("h","coverage"),scale = 0.5, 
           labelPos = "below"
           )

dev.off()

png 
  2

In [ ]:
zoom_bedgraph_enformer <- fread(
  "~/BE_paper_2022/data/enformer_results/CD69.all.bedGraph.all.bedGraph",
  col.names = c('chromosome', 'start', 'end', 'value')
)
zoom_bedgraph_enformer$value <- abs(zoom_bedgraph_enformer$value)

IG_bedgraph <- DataTrack(
  range = zoom_bedgraph_enformer,
  type = c("gradient"),
  genome = 'hg38',
  chromosome=chr_no,
  start=chr_start, # start of region
  end=chr_end, # end of region
  name = "Input*Gradient",
    col="red",
    showColorBar=TRUE
    
)
pdf("cd69_gradient_9.60_9.68.pdf")
plotTracks(list(gtrack,
                #gtTrack,
                IG_bedgraph), shape = "arrow", 
           transcriptAnnotation = "symbol", chromosome = "chr12",from = 9760000, to= 9768000,
           type = c("gradient"),
           ncolor=50,
           gradient=c("white","red")
           )
dev.off()